# Попрактикуемся с тем, что изучили

Будем практиковаться на датасете:
https://www.kaggle.com/c/avito-demand-prediction

Ваша задача:
1. Создать Dataset для загрузки данных (используем только числовые данные)
2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает число показов на основании числовых данных (вы всегда можете нагенерить дополнительных факторов). Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Учить будем на функцию потерь с кагла (log RMSE) - нужно её реализовать
5. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

Вопросы? в личку @Kinetikm

In [308]:
import numpy as np
import torch
import torchvision
import pandas as pd
from torch import nn
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split

In [4]:
train = pd.read_csv('train.csv.zip', compression='zip')

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


In [272]:
train = train.select_dtypes(exclude='object')
train.dropna(inplace=True)
train.head()

,price,item_seq_number,image_top_1,deal_probability
0,400.0,2,1008.0,0.12789
1,3000.0,19,692.0,0.00000
2,4000.0,9,3032.0,0.43177
3,2200.0,286,796.0,0.80323
4,40000.0,3,2264.0,0.20797


In [273]:
class TrainDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transforms=None, **kwargs):
        super().__init__()
        self.dataset = dataset.values
        self.transforms = transforms
    
    def __len__(self):
        return self.dataset.shape[0]
    
    def __getitem__(self, idx):
        item = self.dataset[[idx]]
        if self.transforms:
            item = self.transforms(item)
            item= item.float()
            return item[0][0][:-1], item[0][0][-1]
        return item

In [274]:
my_dataset = TrainDataset(train, transforms=torchvision.transforms.ToTensor())

train, test = train_test_split(my_dataset, random_state=13, test_size = 0.25)

In [305]:
train_loader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=64, shuffle=True)

In [307]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(3, 3)
        self.bn1 = nn.BatchNorm1d(3)
        self.fc2 = nn.Linear(3, 2)
        self.bn2 = nn.BatchNorm1d(2)
        self.fc3 = nn.Linear(2, 1)
        self.bn3 = nn.BatchNorm1d(1)
        self.dp = nn.Dropout(0.2)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dp(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dp(x)
        x = self.fc3(x)
        x = self.bn3(x)
        x = self.relu(x)
        return x

In [284]:
class RMSE(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self, pred, true):
        return torch.sqrt(self.mse(pred, true))

In [310]:

optimizers = [('Adam', torch.optim.Adam(model.parameters(), lr=0.01)),
              ('SGD', torch.optim.SGD(model.parameters(), lr=0.01)),
              ('RMSprop', torch.optim.RMSprop(model.parameters(), lr=0.01))]
loss_func = RMSE()

for opt_name in ['Adam', 'SGD', 'RMSprop']:
    model = Model()
    if opt_name == 'Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    elif opt_name == 'SGD':
        optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    elif opt_name == 'RMSprop':
        optimizer = torch.optim.RMSprop(model.parameters(), lr=0.01)
    for epoch in tqdm_notebook(range(3)):
        for x_train, y_train in train_loader:
            optimizer.zero_grad()
            pred = model(x_train)
            loss = loss_func(pred, y_train)
            loss.backward()
            optimizer.step()
        with torch.no_grad():
            losses = []
            for x_test, y_test in test_loader:
                pred = model(x_test)
                loss = loss_func(pred, y_test)
                losses.append(loss)
            print(f'optimizer {opt_name} epoch {epoch+1} test loss = {np.mean(losses)}')

<ipython-input-310-6842a42e97c8>:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(3)):


/opt/anaconda3/lib/python3.8/site-packages/torch-1.9.0-py3.8-macosx-10.9-x86_64.egg/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/anaconda3/lib/python3.8/site-packages/torch-1.9.0-py3.8-macosx-10.9-x86_64.egg/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([62])) that is different to the input size (torch.Size([62, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/anaconda3/lib/python3.8/site-packages/torch-1.9.0-py3.8-macosx-10.9-x86_64.egg/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([43])) that is different to the input size (torch.Size([43, 1])

optimizer Adam epoch 1 test loss = 0.25598111748695374
optimizer Adam epoch 2 test loss = 0.2560718357563019
optimizer Adam epoch 3 test loss = 0.2560281753540039



optimizer SGD epoch 1 test loss = 0.255891889333725
optimizer SGD epoch 2 test loss = 0.2559383511543274
optimizer SGD epoch 3 test loss = 0.2559061050415039



optimizer RMSprop epoch 1 test loss = 0.2559695839881897
optimizer RMSprop epoch 2 test loss = 0.25605258345603943
optimizer RMSprop epoch 3 test loss = 0.25620365142822266

